In [ ]:
Relax Inc. Data Science Sample Take Home Challenge
This is a practice take home challenge from Relax Inc.

The data is available as two attached CSV files: takehome_user_engagement. csv takehome_users . csv The data has the following two tables:

A user table ( "takehome_users" ) with data on 12,000 users who signed up for the product in the last two years. This table includes:
name: the user's name
object_id: the user's id
email: email address
creation_source: how their account was created. This takes on one of 5 values:
PERSONAL_PROJECTS: invited to join another user's personal workspace
GUEST_INVITE: invited to an organization as a guest (limited permissions)
ORG_INVITE: invited to an organization (as a full member)
SIGNUP: signed up via the website
SIGNUP_GOOGLE_AUTH: signed up using Google Authentication (using a Google email account for their login id)
creation_time: when they created their account
last_session_creation_time: unix timestamp of last login
opted_in_to_mailing_list: whether they have opted into receiving marketing emails
enabled_for_marketing_drip: whether they are on the regular marketing email drip
org_id: the organization (group of users) they belong to
invited_by_user_id: which user invited them to join (if applicable).
A usage summary table ( "takehome_user_engagement" ) that has a row for each day that a user logged into the product.
Defining an "adopted user" as a user who has logged into the product on three separate days in at least one sevenday period , identify which factors predict future user adoption .

We suggest spending 12 hours on this, but you're welcome to spend more or less. Please send us a brief writeup of your findings (the more concise, the better no more than one page), along with any summary tables, graphs, code, or queries that can help us understand your approach. Please note any factors you considered or investigation you did, even if they did not pan out. Feel free to identify any further research or data you think would be valuable.

In [1]:
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True);sns.set(font_scale=1.2)
import numpy as np
from collections import Counter
# Setup Seaborn
sns.set_style("whitegrid")
sns.set_context("poster")

In [2]:
df_eg=pd.read_csv("takehome_user_engagement.csv")

In [3]:
df_eg.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [4]:
df_eg.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [5]:
df_user=pd.read_csv("takehome_users.csv",encoding = "ISO-8859-1")

In [6]:
df_user.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [7]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


In [8]:
df_eg.time_stamp = pd.to_datetime(df_eg.time_stamp)
df_eg = df_eg.set_index('time_stamp', drop= True)

In [9]:
df_temp = df_eg.resample('D').mean().dropna()

In [10]:
df_temp

,user_id,visited
time_stamp,,
2012-05-31,6258.000000,1.0
2012-06-01,8113.333333,1.0
2012-06-02,6787.222222,1.0
2012-06-03,5539.727273,1.0
2012-06-04,5398.818182,1.0
...,...,...
2014-06-01,5824.700000,1.0
2014-06-02,6042.183168,1.0
2014-06-03,5783.198492,1.0


In [11]:
df_agg_d = df_eg.groupby([pd.Grouper(freq='D'),'user_id']).mean().dropna()

In [12]:
df_agg_d=df_agg_d.reset_index()


In [13]:
df_agg_d=df_agg_d.sort_values(by=['user_id','time_stamp']).reset_index()

In [14]:
## find outadopted user"  who has logged into the product on three separate days in at least one sevenday period 
from datetime import timedelta
adopted=[]

for i in range(df_agg_d.shape[0]-2):
    if df_agg_d.user_id[i+2]==df_agg_d.user_id[i] and df_agg_d.time_stamp[i + 2] - df_agg_d.time_stamp[i] <= timedelta(days=7):
        adopted.append(df_agg_d.user_id[i])

In [15]:
myset=set(adopted)

In [49]:
len(myset)

1656

In [ ]:
## There are only 1656 active users in the dataframe,
## about 14% of the total users. Keep this in mind later since it means that the classes will need to be balanced later for analysis.

In [ ]:
### Merge the adopted_user id with the big data

In [16]:
user_id_o=[item for item in myset]

In [17]:
user_id_o=sorted(user_id_o)

In [18]:
user_id_ado=pd.DataFrame(user_id_o)

In [19]:
user_id_ado['adopted_user']=1

In [20]:
user_id_ado=user_id_ado.rename(columns={0:'id'})

In [21]:
user_mer=pd.merge(df_user,user_id_ado,left_on='object_id',right_on='id',how='outer')

In [22]:
user_mer.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,id,adopted_user
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,NaN,NaN
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2.0,1.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,NaN,NaN
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,NaN,NaN
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,NaN,NaN


In [23]:
user_mer.adopted_user=user_mer.adopted_user.fillna(0)

In [24]:
user_mer.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,id,adopted_user
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,NaN,0.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0,2.0,1.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0,NaN,0.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0,NaN,0.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0,NaN,0.0


In [25]:
# set creation_time
import datetime
user_mer.creation_time = pd.to_datetime(user_mer.creation_time)
#last_session_creation_time is a unix time stamp, so use map to convert to datetime
user_mer['last_session_creation_time'] = user_mer['last_session_creation_time'].map(lambda data: 
                                    datetime.datetime.fromtimestamp(int(data)).strftime('%Y-%m-%d %H:%M:%S'),
                                                                              na_action='ignore')

In [26]:
user_mer.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,id,adopted_user
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-21 23:53:30,1,0,11,10803.0,NaN,0.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-30 23:45:04,0,0,1,316.0,2.0,1.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 19:14:52,0,0,94,1525.0,NaN,0.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 04:09:28,0,0,1,5151.0,NaN,0.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 05:14:20,0,0,193,5240.0,NaN,0.0


In [27]:
#Creat a usage_length feature
#now set that to datetime
user_mer['last_session_creation_time'] = pd.to_datetime(user_mer['last_session_creation_time'])
#subtract to find time active
user_mer['usage_length'] = abs(user_mer['last_session_creation_time'] - user_mer['creation_time'])
#lets settle for seconds instead of days to make the time differences more distinct
user_mer['usage_length'] = [x.total_seconds() for x in user_mer['usage_length']]

In [28]:
user_mer.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,id,adopted_user,usage_length
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,2014-04-21 23:53:30,1,0,11,10803.0,NaN,0.0,14400.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,2014-03-30 23:45:04,0,0,1,316.0,2.0,1.0,11736000.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,2013-03-19 19:14:52,0,0,94,1525.0,NaN,0.0,14400.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,2013-05-22 04:09:28,0,0,1,5151.0,NaN,0.0,72000.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,2013-01-22 05:14:20,0,0,193,5240.0,NaN,0.0,414000.0


In [37]:
feature_df = user_mer.iloc[:,4:]
feature_df = feature_df.drop(['last_session_creation_time','id','invited_by_user_id'], axis=1)
feature_df['usage_length'] = feature_df['usage_length'].fillna(0)

In [38]:
feature_df.head()

,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted_user,usage_length
0,GUEST_INVITE,1,0,11,0.0,14400.0
1,ORG_INVITE,0,0,1,1.0,11736000.0
2,ORG_INVITE,0,0,94,0.0,14400.0
3,GUEST_INVITE,0,0,1,0.0,72000.0
4,GUEST_INVITE,0,0,193,0.0,414000.0


In [39]:
from sklearn.preprocessing import LabelEncoder
gle = LabelEncoder()
creation_labels = gle.fit_transform(user_mer['creation_source'])
feature_df.creation_source = creation_labels

In [40]:
feature_df.head()

,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted_user,usage_length
0,0,1,0,11,0.0,14400.0
1,1,0,0,1,1.0,11736000.0
2,1,0,0,94,0.0,14400.0
3,0,0,0,1,0.0,72000.0
4,0,0,0,193,0.0,414000.0


In [41]:
for col in feature_df.columns:
    feature_df[col] = feature_df[col].astype('float64')
feature_df.fillna(0)

,creation_source,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,adopted_user,usage_length
0,0.0,1.0,0.0,11.0,0.0,14400.0
1,1.0,0.0,0.0,1.0,1.0,11736000.0
2,1.0,0.0,0.0,94.0,0.0,14400.0
3,0.0,0.0,0.0,1.0,0.0,72000.0
4,0.0,0.0,0.0,193.0,0.0,414000.0
...,...,...,...,...,...,...
11995,1.0,0.0,0.0,89.0,0.0,14400.0
11996,4.0,0.0,0.0,200.0,0.0,414000.0
11997,0.0,1.0,1.0,83.0,0.0,14400.0
11998,2.0,0.0,0.0,6.0,0.0,158400.0


In [42]:
feature_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12000 entries, 0 to 11999
Data columns (total 6 columns):
creation_source               12000 non-null float64
opted_in_to_mailing_list      12000 non-null float64
enabled_for_marketing_drip    12000 non-null float64
org_id                        12000 non-null float64
adopted_user                  12000 non-null float64
usage_length                  12000 non-null float64
dtypes: float64(6)
memory usage: 656.2 KB


In [43]:
from sklearn.model_selection import train_test_split

#set up data by seperating out the labels, then split for cross validation
data = feature_df.drop('adopted_user', axis=1)
labels = feature_df.adopted_user

X_train, y_train, X_test, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [45]:
from sklearn.ensemble import RandomForestClassifier

#train and test classifier
rf = RandomForestClassifier(class_weight='balanced_subsample')

rf.fit(X_train, X_test)

rf.score(y_train, y_test)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.97125

In [46]:
from sklearn.metrics import classification_report, confusion_matrix

#print out classification report and confusion matrix
y_pred = rf.predict(y_train)

print(classification_report(y_test, y_pred))

cm= confusion_matrix(y_test,y_pred)
print('confusion matrix:')
print(cm)

              precision    recall  f1-score   support

         0.0       0.98      0.99      0.98      2063
         1.0       0.93      0.86      0.89       337

    accuracy                           0.97      2400
   macro avg       0.95      0.92      0.94      2400
weighted avg       0.97      0.97      0.97      2400

confusion matrix:
[[2042   21]
 [  48  289]]


In [47]:
#make a df that displays the cofficients indexed by feature name
feature_importance = pd.DataFrame()
feature_importance['coef'] = rf.feature_importances_
feature_importance = feature_importance.set_index(data.columns)
feature_importance.coef.nlargest(10)

usage_length                  0.919760
org_id                        0.060881
creation_source               0.012634
opted_in_to_mailing_list      0.004322
enabled_for_marketing_drip    0.002403
Name: coef, dtype: float64

Based on the above analysis, RandomForestClassifer identified usage_length (how long between signing up and last active session) to be the greatest indicator of whether or not they became an adopted user. Lets see what that means in a post hoc visualization